In [1]:
import tkinter
from PIL import Image,ImageTk
from tkinter import *
from tkinter import messagebox
import os
import cv2
import csv
import numpy as np
import pandas as pd
import datetime
import time
from threading import Thread
import yagmail
import smtplib
import imaplib
from PIL import Image

In [2]:
#QUIT Function
def quit_window():
    if messagebox.askyesno("Quit", "You are exiting window.Do you want to quit?"):
        window.destroy()
    
#contact
def contact():
    messagebox._show(title="Contact Me",message="If you find you need any help contact me on 'riyasnegi12@gmail.com'!")
    
#about
def about():
    messagebox._show(title="About",message="This Attendance Based System is designed by Riya Negi!")

In [3]:
#Function to check Webcam

def webcamCheck():
    import cv2
    # Load the Haar cascade classifier
    face_cascade = cv2.CascadeClassifier('E:/FRAS/haarcascade_frontalface_default.xml')
    # To capture video from webcam.
    capture = cv2.VideoCapture(0)
    while True:
        # Read the frame
        _, img = capture.read()
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Detect the faces
        faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(30, 30),flags = cv2.CASCADE_SCALE_IMAGE)
        # Draw the rectangle around each face
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (10,159,255), 2)
        cv2.imshow('Webcam Check', img)
        # Stop if escape key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # Release the VideoCapture object
    capture.release()
    cv2.destroyAllWindows()

In [4]:
#Login to the mail account through IMAP and SMTP

imap = imaplib.IMAP4_SSL('imap.gmail.com',993)                                          
unm = 'marked.attendance@gmail.com'                                                      
psw = 'mogikhmfjckhxyfw'                                                                 
imap.login(unm,psw) 

mail=smtplib.SMTP('smtp.gmail.com',587)
mail.ehlo()
mail.starttls()
mail.login(unm,psw)

(235, b'2.7.0 Accepted')

In [5]:
#Function for the member window

def memberWindow():

    def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            pass
        try:
            import unicodedata
            unicodedata.numeric(s)
            return True
        except (TypeError, ValueError):
            pass
        return False

    # Capture Face Function for creating dataset
    def captureFace():
        def submit():
            Id=Id_var.get()
            name=Name_var.get()
            Name_var.set("")
            Id_var.set("")
            if(is_number(Id) and name.isalpha()):
                cam = cv2.VideoCapture(0)
                harcascadePath = "E:/FRAS/haarcascade_frontalface_default.xml"
                detector = cv2.CascadeClassifier(harcascadePath)
                sampleNum = 0
                while(True):
                    ret, img = cam.read()
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    faces = detector.detectMultiScale(gray, 1.3, 5, minSize=(30,30),flags = cv2.CASCADE_SCALE_IMAGE)
                    for(x,y,w,h) in faces:
                        cv2.rectangle(img, (x, y), (x+w, y+h), (10, 159, 255), 2)
                        #incrementing sample number
                        sampleNum = sampleNum+1
                        #saving the captured face in the dataset folder TrainingImages
                        cv2.imwrite("E:/FRAS/TrainingImages" + os.sep +name + "."+Id + '.' +
                                    str(sampleNum) + ".jpg", gray[y:y+h, x:x+w])
                        #display the frame
                        cv2.imshow('frame', img)
                    #wait for 100 miliseconds
                    if cv2.waitKey(100) & 0xFF == ord('q'):
                        break
                    # break if the sample number is more than 100
                    elif sampleNum > 100:
                        break
                loginframe.destroy()
                cam.release()
                cv2.destroyAllWindows()
                #res = "Images Saved for ID : " + Id + " Name : " + name
                header=["Id", "Name"]
                row = [Id, name]
                if(os.path.isfile("E:/FRAS/StudentDetails/StudentDetails.csv")):
                    with open("E:/FRAS/StudentDetails/StudentDetails.csv", 'a+') as csvFile:
                        writer = csv.writer(csvFile)
                        writer.writerow(j for j in row)
                    csvFile.close()
                else:
                    with open("E:/FRAS/StudentDetails/StudentDetails.csv", 'a+') as csvFile:
                        writer = csv.writer(csvFile)
                        writer.writerow(i for i in header)
                        writer.writerow(j for j in row)
                    csvFile.close()
            else:
                if(is_number(Id)):
                    messagebox.showwarning("Warning","Enter Alphabetical Name without white spaces or numbers!",parent=Memberwindow)
                    captureFace()
                if(name.isalpha()):
                    messagebox.showwarning("Warning","Enter Numeric ID!",parent=Memberwindow)
                    captureFace()
                    
        loginframe = tkinter.Frame(Memberwindow, bg="black")
        loginframe.place(relx=0.5, rely=0.23, relwidth=0.40, relheight=0.52)
        IDLabel = Label(loginframe, text="Enter your ID",fg="white", bg="#355454", width=11,font=('times', 20, ' bold '))
        IDLabel.place(relx=0.05, rely=0.25)
        Id_var = StringVar()
        IDEntry = Entry(loginframe, textvariable=Id_var, font=('times', 15, ' bold '))
        IDEntry.place(relx=0.5, rely=0.25,relwidth=0.45, relheight=0.1)
        NAMELabel = Label(loginframe,text="Enter your Name",fg="white", bg="#355454", width=13,font=('times', 20, ' bold '))
        NAMELabel.place(relx=0.04, rely=0.5)
        Name_var = StringVar()
        NAMEEntry = Entry(loginframe, textvariable=Name_var, font=('times', 15, ' bold '))
        NAMEEntry.place(relx=0.5, rely=0.5,relwidth=0.45, relheight=0.1)  
        SubmitButton = Button(loginframe, text="Submit",command=submit,font=('times', 20, ' bold '))
        SubmitButton.place(relx=0.45, rely=0.75)
       
    #Function to get Images and labels
    def getImagesAndLabels(path):
        # get the path of all the files in the folder
        imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
        # create empty face list
        faces = []
        # create empty ID list
        Ids = []
        # now looping through all the image paths and loading the Ids and the images of the face
        for imagePath in imagePaths:
            # loading the image and converting it to gray scale
            pilImage = Image.open(imagePath).convert('L')
            # Now we are converting the PIL image into numpy array
            imageNp = np.array(pilImage, 'uint8')
            # getting the Id from the image
            Id = int(os.path.split(imagePath)[-1].split(".")[1])
            # extract the face from the training image sample
            faces.append(imageNp)
            Ids.append(Id)
        return faces, Ids


    #Function to train images saved in the desired file location
    def TrainImages():
        recognizer = cv2.face_LBPHFaceRecognizer.create()
        harcascadePath = "E:/FRAS/haarcascade_frontalface_default.xml"
        detector = cv2.CascadeClassifier(harcascadePath)
        faces, Id = getImagesAndLabels("E:/FRAS/TrainingImages")
        Thread(target = recognizer.train(faces, np.array(Id))).start()
        recognizer.save("E:/FRAS/TrainingImageLabel"+os.sep+"Trainner.yml")
        messagebox.showinfo("Images Trained", "Images Successfully Trained!",parent=Memberwindow)

    #Function to recognize the face for the attendance
    def recognize_attendence():
        recognizer = cv2.face.LBPHFaceRecognizer_create()
        recognizer.read("E:/FRAS/TrainingImageLabel/Trainner.yml")
        harcascadePath = "E:/FRAS/haarcascade_frontalface_default.xml"
        faceCascade = cv2.CascadeClassifier(harcascadePath)
        df = pd.read_csv("E:/FRAS/StudentDetails"+os.sep+"StudentDetails.csv")
        font = cv2.FONT_HERSHEY_SIMPLEX
        col_names = ['Id', 'Name', 'Date', 'Time']
        attendance = pd.DataFrame(columns=col_names)

        # Initialize and start realtime video capture
        cam = cv2.VideoCapture(0)
        cam.set(3, 640)  # set video width
        cam.set(4, 480)  # set video height
        # Define min window size to be recognized as a face
        minW = 0.1 * cam.get(3)
        minH = 0.1 * cam.get(4)

        while True:
            _,im = cam.read()
            gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            faces = faceCascade.detectMultiScale(gray, 1.2, 5,minSize = (int(minW), int(minH)),flags = cv2.CASCADE_SCALE_IMAGE)
            for(x, y, w, h) in faces:
                cv2.rectangle(im, (x, y), (x+w, y+h), (10, 159, 255), 2)
                Id, conf = recognizer.predict(gray[y:y+h, x:x+w])
                if conf < 100:
                    aa = df.loc[df['Id'] == Id]['Name'].values
                    confstr = "  {0}%".format(round(100 - conf))
                    tt = str(Id)+"-"+aa
                else:
                    Id = '  Unknown  '
                    tt = str(Id)
                    confstr = "  {0}%".format(round(100 - conf))

                if (100-conf) > 67:
                    ts = time.time()
                    date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
                    timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                    aa = str(aa)[2:-2]
                    attendance.loc[len(attendance)] = [Id, aa, date, timeStamp]

                tt = str(tt)[2:-2]
                if(100-conf) > 67:
                    tt = tt + " [Pass]"
                    cv2.putText(im, str(tt), (x+5,y-5), font, 1, (255, 255, 255), 2)
                else:
                    cv2.putText(im, str(tt), (x + 5, y - 5), font, 1, (255, 255, 255), 2)

                if (100-conf) > 67:
                    cv2.putText(im, str(confstr), (x + 5, y + h - 5), font,1, (0, 255, 0),1 )
                elif (100-conf) > 50:
                    cv2.putText(im, str(confstr), (x + 5, y + h - 5), font, 1, (0, 255, 255), 1)
                else:
                    cv2.putText(im, str(confstr), (x + 5, y + h - 5), font, 1, (0, 0, 255), 1)

            attendance = attendance.drop_duplicates(subset=['Id'], keep='first')
            cv2.imshow('Attendance', im)
            if (cv2.waitKey(1) == ord('q')):
                break
        ts = time.time()
        date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
        timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
        Hour, Minute, Second = timeStamp.split(":")
        fileName = "E:/FRAS/Attendance"+os.sep+"Attendance_"+date+"_"+Hour+"-"+Minute+".csv"
        attendance.to_csv(fileName, index=False)
        messagebox.showinfo("Attendance Successful", "Attendance Successful!",parent=Memberwindow)
        cam.release()
        cv2.destroyAllWindows()
    Memberwindow = Toplevel()
    Memberwindow.title("Member")
    Memberwindow.geometry("1280x720")
    Memberwindow.resizable(True,True)
    Memberwindow.configure(background='#355454')

    message = tkinter.Label(Memberwindow, text="Member" ,fg="white",bg="#355454" ,width=60 ,height=1,font=('times', 29, ' bold '))
    message.place(x=5, y=20,relwidth=1)
         
    img1 = Image.open("E:/FRAS/FRAS/Member.png")     
    resized_image= img1.resize((400,350), Image.ANTIALIAS)
    new_image= ImageTk.PhotoImage(resized_image)
    label1 = Label(Memberwindow, image=new_image)
    label1.image = new_image
    label1.place(relx=0.05,rely=0.23)
    frame1 = tkinter.Frame(Memberwindow, bg="PaleTurquoise4")
    frame1.place(relx=0.5, rely=0.23, relwidth=0.40, relheight=0.52)

    check = tkinter.Button(frame1, text="Check WebCam", command= webcamCheck , fg="black", bg="light grey", activebackground = "white", font=('times', 20, ' bold '))
    check.place(x=130, y=30,relwidth=0.55)
    cap_faces = tkinter.Button(frame1, text="Capture Your Face", command=captureFace, fg="black", bg="light grey", activebackground = "white", font=('times', 20, ' bold '))
    cap_faces.place(x=130, y=110,relwidth=0.55)
    train = tkinter.Button(frame1, text="Train Your Images", command=TrainImages , fg="black", bg="light grey", activebackground = "white", font=('times', 20, ' bold '))
    train.place(x=130, y=190,relwidth=0.55)
    recog_att = tkinter.Button(frame1, text="Click For Attendance", command=recognize_attendence , fg="black", bg="light grey", activebackground = "white", font=('times', 20, ' bold '))
    recog_att.place(x=130, y=270,relwidth=0.55)
    def homewindow():
        Memberwindow.destroy()
    HomeButton = Button(Memberwindow, text="Home", font=('times', 20, ' bold '),command=homewindow)
    HomeButton.place(relx=0.67, rely=0.8)

In [6]:
#Function for the Admin window

def admin():
    def submit():
        def mailsubmit():
            recipient=recEntry.get()
            date = datetime.date.today().strftime("%B %d, %Y")
            path = 'E:/FRAS/Attendance'
            os.chdir(path)
            files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)
            newest = files[-1]
            filename = newest
            sub = "Attendance Report for " + str(date)
            yag = yagmail.SMTP(unm, psw)
            yag.send(
                to=recipient, #recipient email ID
                subject=sub, # email subject
                contents="Attendance Details of the Company",  # email body
                attachments= filename  # file attached
            )
            messagebox.showinfo("Email Sent", "Email Successfully Sent!",parent=mailwindow)
        username=usernameEntry.get()
        password=passwordEntry.get()
        if (username==unm):
            if (password==psw):
                Adminwindow.destroy()
                mailwindow = Toplevel()
                mailwindow.title("AutoMail")
                mailwindow.geometry("1280x720")
                mailwindow.resizable(True,True)
                mailwindow.configure(background='#355454')
                message = tkinter.Label(mailwindow, text="AutoMail Attendance" ,fg="white",bg="#355454" ,width=60 ,height=1,font=('times', 29, ' bold '))
                message.place(x=5, y=20,relwidth=1)
                img1 = Image.open("E:/FRAS/FRAS/Mail.png")     
                resized_image= img1.resize((400,350), Image.ANTIALIAS)
                new_image= ImageTk.PhotoImage(resized_image)
                label1 = Label(mailwindow, image=new_image)
                label1.image = new_image
                label1.place(relx=0.05,rely=0.23)        
                frame2 = tkinter.Frame(mailwindow, bg="#355454")
                frame2.place(relx=0.5, rely=0.23, relwidth=0.40, relheight=0.52)
                gidLabel = Label(frame2,text="Enter Email ID",fg="white", bg="#355454", width=11,font=('times', 20, ' bold '))
                gidLabel.place(relx=0.05, rely=0.1)
                rec = StringVar()
                recEntry = Entry(frame2, textvariable=rec, font=('times', 15, ' bold '))
                recEntry.place(relx=0.45, rely=0.1,relwidth=0.5, relheight=0.1)
                automail1 = tkinter.Button(frame2, text="AutoMail Attendance", command=mailsubmit,fg="black", bg="light grey", width=20, activebackground = "white", font=('times', 20, ' bold '))
                automail1.place(relx=0.3, rely=0.4,relwidth=0.65,relheight=0.15)
                def homewindow():
                    mailwindow.destroy()
                HomeButton = Button(frame2, text="Home", font=('times', 20, ' bold '),command=homewindow)
                HomeButton.place(relx=0.55, rely=0.7)
            else :
                messagebox.showwarning("Warning","Enter Correct Password!",parent=Adminwindow)
        else :
            messagebox.showwarning("Warning","Enter Correct Username!",parent=Adminwindow)
            
        
    Adminwindow = Toplevel()
    Adminwindow.title("Admin")
    Adminwindow.geometry("1280x720")
    Adminwindow.resizable(True,True)
    Adminwindow.configure(background='#355454')
    message = tkinter.Label(Adminwindow, text="Admin Login" ,fg="white",bg="#355454" ,width=60 ,height=1,font=('times', 29, ' bold '))
    message.place(x=5, y=20,relwidth=1)
    img1 = Image.open("E:/FRAS/FRAS/Admin.png")     
    resized_image= img1.resize((400,350), Image.ANTIALIAS)
    new_image= ImageTk.PhotoImage(resized_image)
    label1 = Label(Adminwindow, image=new_image)
    label1.image = new_image
    label1.place(relx=0.05,rely=0.23) 

    loginframe = tkinter.Frame(Adminwindow, bg="black")
    loginframe.place(relx=0.5, rely=0.23, relwidth=0.40, relheight=0.52)
    usernameLabel = Label(loginframe, text="User Name",fg="white", bg="#355454", width=11,font=('times', 20, ' bold '))
    usernameLabel.place(relx=0.05, rely=0.25)
    username = StringVar()
    usernameEntry = Entry(loginframe, textvariable=username, font=('times', 15, ' bold '))
    usernameEntry.place(relx=0.45, rely=0.25,relwidth=0.50, relheight=0.1)
    passwordLabel = Label(loginframe,text="Password",fg="white", bg="#355454", width=11,font=('times', 20, ' bold '))
    passwordLabel.place(relx=0.05, rely=0.5)
    password = StringVar()
    passwordEntry = Entry(loginframe, textvariable=password, show='*', font=('times', 15, ' bold '))
    passwordEntry.place(relx=0.45, rely=0.5,relwidth=0.50, relheight=0.1)  
    loginButton = Button(loginframe, text="Login",command=submit, font=('times', 20, ' bold '))
    loginButton.place(relx=0.45, rely=0.75)
    def homewindow():
        Adminwindow.destroy()
    HomeButton = Button(Adminwindow, text="Home", font=('times', 20, ' bold '),command=homewindow)
    HomeButton.place(relx=0.68, rely=0.8)

In [7]:
#Function for the main window

def mainWindow():
    #Help menubar----------------------------------------------
    menubar=Menu(window)
    help=Menu(menubar,tearoff=0)
    help.add_command(label="Contact Us",command=contact)
    help.add_separator()
    help.add_command(label="Exit",command=quit_window)
    menubar.add_cascade(label="Help",menu=help)
    menubar.add_command(label="About",command=about)
    window.config(menu=menubar)
    message3 = tkinter.Label(window, text="Face Recognition Based Attendance System" ,fg="white",bg="#355454" ,width=60 ,height=1,font=('times', 29, ' bold '))
    message3.place(x=5, y=20,relwidth=1)

    canvas = Canvas(window, width = 500, height = 350)      
    canvas.place(relx=0.05, rely=0.23)      
    img = PhotoImage(file="E:/FRAS/FRAS/MainWindow.png")      
    canvas.create_image(20,20, anchor=NW, image=img)        
    frame1 = tkinter.Frame(window, bg="PaleTurquoise4")
    frame1.place(relx=0.5, rely=0.23, relwidth=0.40, relheight=0.52)

    adminButton = tkinter.Button(frame1, text="Admin",command=admin,fg="black", bg="light grey", width=11, activebackground = "white", font=('times', 20, ' bold '))
    adminButton.place(x=170, y=50,relwidth=0.35,relheight=0.15)

    member = tkinter.Button(frame1, text="Member", fg="black", command=memberWindow, bg="light grey", width=11, activebackground = "white", font=('times', 20, ' bold '))
    member.place(x=170, y=150,relwidth=0.35,relheight=0.15)
    
    quit = tkinter.Button(frame1, text="Quit", command=quit_window, fg="black", bg="light grey", width=11, activebackground = "white", font=('times', 20, ' bold '))
    quit.place(x=170, y=250,relwidth=0.35,relheight=0.15)
    
    window.protocol("WM_DELETE_WINDOW", quit_window)
    window.mainloop()


In [8]:
window = tkinter.Tk()
window.title("Face Recognition Based Attendance System")
window.geometry("1280x720")
window.resizable(True,True)
window.configure(background='#355454')
mainWindow()